# Group B4 Assignment-2 submission 

In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import re

import time
from tqdm import tqdm
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
opts=Options()
opts.add_argument('user-agent = Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6)')

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options = opts)

/var/folders/4_/22qxnscx4xd9pm5c0lzrhzyh0000gn/T/ipykernel_7626/2578296226.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options = opts)
/var/folders/4_/22qxnscx4xd9pm5c0lzrhzyh0000gn/T/ipykernel_7626/2578296226.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options = opts)


In [3]:
##This piece of code here enables interacting with Selenium web driver and logs the user into
##his/her LinkedIn account

#driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
driver.get("https://www.linkedin.com/login")    #Loading Linkedin's login page

time.sleep(2)

#Accepting cookies on login page
xpath="""//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]"""
driver.find_element("xpath",xpath).click()

#Entering credentials of the linkedin account
xpath_username="""//*[@id="username"]"""
xpath_password="""//*[@id="password"]"""
username="a.e.schwendinger@gmail.com"
password="Anne1995"
driver.find_element("xpath",xpath_username).send_keys(username)
driver.find_element("xpath",xpath_password).send_keys(password)
time.sleep(1)

#Signing in to the account 
xpath_signin="""//*[@id="organic-div"]/form/div[3]/button"""
driver.find_element("xpath",xpath_signin).click()
driver.implicitly_wait(30)

In [4]:
#This piece of code is used to get the URLs of all the 40 pages of our interest
base_url="https://www.linkedin.com/jobs/search/?currentJobId=3270287326&geoId=107025191&keywords=data%20scientist%2C%20barcelona&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true"

pages=[]
for i in range(40):
    pages.append(base_url+"&start="+str(i*25))
    

# Main block of code used for Web Scraping

In [5]:
#Defining the required empty lists
texts=[]
right=[]

job_title=[]
company_name=[]
location=[]
workspace=[]
promoted=[]
app_through_linkedin=[]
state=[]
posting_date=[]
applicants=[]
python_req=[]
employment_type=[]
seniority=[]
employees=[]
industry=[]
urls=[]


for page in tqdm(pages):    #This is the over-arching loop which loops job pages
    #Going to the required job URL page
    driver.get(page)
    time.sleep(1)         #waiting for 1 second
    
    
    jobs_block = driver.find_element(By.CLASS_NAME, "scaffold-layout__list-container")
    jobs=driver.find_elements(By.CSS_SELECTOR, ".jobs-search-results__list-item")
    

    for job in jobs:     #This for loop loops over all the jobs present in the page
        try:
            if str(job.find_element(By.TAG_NAME,"a").get_attribute('href')).startswith("https://www.linkedin.com/jobs/view"):
                try:
                    
                    job.click()       #Clicking on a particular job on the left panel of the page
                                      #so that information of the job appears on the right  
                    texts.append(job.text)
                    
                    time.sleep(0.8)     #waiting for 0.8 seconds
                    right.append(driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__content--two-pane").text)

                    
                    #Code for offer_urls
                    urls.append(driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__content--two-pane").find_element(By.TAG_NAME,"a").get_attribute("href"))
                    
                    
                    #Code for State
                    state_combs=["actively hiring", "actively recruiting", "early applicant"]
                    state_combs_dic={"actively hiring":"On-going","actively recruiting":"On-going",
                                     "early applicant":"Early Applications"}

                    check=0
                    index=[]

                    for i in state_combs:
                        if i in right[jobs.index(job)].lower():
                            check+=1
                            index.append(state_combs.index(i))
                    if check==0:
                        state.append(np.nan)
                    elif check==1:
                        state.append(state_combs_dic[state_combs[index[0]]])


                    #Code for posting date
                    pattern="([\d]+) (minutes ago|minute ago|hour ago|hours ago|day ago|days ago|week ago|weeks ago|month ago|months ago)"
                    posting_date.append(re.findall(pattern,right[jobs.index(job)])[0][0]+" "+re.findall(pattern,right[jobs.index(job)])[0][1])


                    #Code for applicants
                    pattern="[\d]+ applicants"
                    if "applicants" in right[jobs.index(job)]:
                        applicants.append(int(re.findall(pattern,right[jobs.index(job)])[0].strip().split(" ")[0]))
                    else:
                        applicants.append(np.nan)


                    #Code for python_req
                    if "python" in driver.find_element(By.TAG_NAME, "body").text.lower():
                        python_req.append(True)
                    else:
                        python_req.append(False)


                    #Code for employment_type
                    employment_type.append(right[jobs.index(job)].split("\n")[2].split("·")[0].strip())


                    #Code for seniority
                    if len(right[jobs.index(job)].split("\n")[2].split("·"))==2:
                        seniority.append(right[jobs.index(job)].split("\n")[2].split("·")[1].strip())
                    else:
                        seniority.append(np.nan)


                    #Code for employees and industry
                    emp_ind=-1
                    for i in right[jobs.index(job)].split("\n"):
                        if "employees" in i:
                            emp_ind=i
                            break
                    if emp_ind==-1:
                        employees.append(np.nan)
                        industry.append(np.nan)
                    else:
                        if len(emp_ind.split("·"))==2:
                            employees.append(emp_ind.split("·")[0].strip().split(" ")[0].strip())
                            industry.append(emp_ind.split("·")[1].strip())
                        elif len(emp_ind.split("·"))==1:
                            employees.append(emp_ind.split("·")[0].strip().split(" ")[0].strip())
                            industry.append(np.nan)

                    #Code for jon title, company name and location
                    job_title.append(texts[jobs.index(job)].split("\n")[0].strip())
                    company_name.append(texts[jobs.index(job)].split("\n")[1].strip())
                    location.append(texts[jobs.index(job)].split("\n")[2].strip())
                    
                    
                    #Code for Mode of work
                    mode_of_work=["Hybrid","On-site","Remote"]
                    check=0
                    index=[]
                    for i in mode_of_work:
                        if i in texts[jobs.index(job)]:
                            check+=1
                            index.append(mode_of_work.index(i))
                    if check==0:
                        workspace.append(np.nan)
                    elif check==1:
                        workspace.append(mode_of_work[index[0]])
                    else:
                        workspace.append("Other")
                    
                    
                    #Code for Promoted
                    if "Promoted" in texts[jobs.index(job)]:
                        promoted.append(True)
                    else:
                        promoted.append(False)
                    
                    #Code for Easy apply
                    if "Easy Apply" in texts[jobs.index(job)]:
                        app_through_linkedin.append(True)
                    else:
                        app_through_linkedin.append(False)
                    
                    #Scrolling through the jobs page
                    driver.execute_script("arguments[0].scrollIntoView();", job)  
                    
                    
                except:       
                    job_title.append(np.nan)
                    company_name.append(np.nan)
                    location.append(np.nan)
                    promoted.append(np.nan)
                    workspace.append(np.nan)
                    app_through_linkedin.append(np.nan)
                    state.append(np.nan)
                    posting_date.append(np.nan)
                    applicants.append(np.nan)
                    python_req.append(np.nan)
                    employment_type.append(np.nan)
                    seniority.append(np.nan)
                    employees.append(np.nan)
                    industry.append(np.nan)

            else:
                job_title.append(np.nan)
                company_name.append(np.nan)
                location.append(np.nan)
                promoted.append(np.nan)
                workspace.append(np.nan)
                app_through_linkedin.append(np.nan)
                state.append(np.nan)
                posting_date.append(np.nan)
                applicants.append(np.nan)
                python_req.append(np.nan)
                employment_type.append(np.nan)
                seniority.append(np.nan)
                employees.append(np.nan)
                industry.append(np.nan)
        except:
            job_title.append(np.nan)
            company_name.append(np.nan)
            location.append(np.nan)
            promoted.append(np.nan)
            workspace.append(np.nan)
            app_through_linkedin.append(np.nan)
            state.append(np.nan)
            posting_date.append(np.nan)
            applicants.append(np.nan)
            python_req.append(np.nan)
            employment_type.append(np.nan)
            seniority.append(np.nan)
            employees.append(np.nan)
            industry.append(np.nan)
            
            

100%|███████████████████████████████████████████| 40/40 [24:06<00:00, 36.17s/it]


In [13]:
#Between pages, the driver is redirected to other links because of broken redirections in few of the last pages.
#Hence there are few extra NAs in all the lists except for the URL because the particular tag is not found in the 
#other pages.

#Getting the extra indices
indices=[]
for i in range(len(job_title)): 
    if job_title[i] is np.nan:
        indices.append(i)

In [7]:
#Removing unnecessary NAs

for i in indices:
    del job_title[i]
    del company_name[i]
    del location[i]
    del promoted[i]
    del workspace[i]
    del app_through_linkedin[i]
    del state[i]
    del posting_date[i]
    del applicants[i]
    del python_req[i]
    del employment_type[i]
    del seniority[i]
    del employees[i]
    del industry[i]

In [8]:
#Changing the format of the Number of employees column
#Wherever there is a range for employees -> we have considered an average of the interval's extreme values
updated_employees=[]
for e in employees:
    if e is not np.nan:
        if e == "10,001+":
            updated_employees.append(10000)
        else:
            updated_employees.append(int((int(e.split("-")[0].replace(",",""))+int(e.split("-")[1].replace(",","")))/2))
    else:
        updated_employees.append(np.nan)
        

In [9]:
#Creating a dictionary to pass it to the final dataframe
dic={"Job Title":job_title, "Company Name":company_name, "Location":location,
     "Offer URL":urls, "Promoted":promoted, "Workspace":workspace,
     "Application through Linkedin":app_through_linkedin, "State":state,
    "Posting Date":posting_date, "Number of Applicants":applicants,
    "Python Required":python_req,"Employment Type":employment_type,"Seniority":seniority,
    "Number of Employees":updated_employees,"Industry":industry}

#Creating the final dataframe 
df=pd.DataFrame(dic)

In [10]:
#Converting Number of applicants to Integers
df["Number of Applicants"] = df["Number of Applicants"].astype('Int64')

In [11]:
#Converting Number of Employees	 to Integers
df["Number of Employees"] = df["Number of Employees"].astype('Int64')

In [12]:
#Printing the dataframe
df

,Job Title,Company Name,Location,Offer URL,Promoted,Workspace,Application through Linkedin,State,Posting Date,Number of Applicants,Python Required,Employment Type,Seniority,Number of Employees,Industry
0,System Integrator,Bunge,"Sant Just Desvern, Catalonia, Spain",https://www.linkedin.com/jobs/view/3302875305/...,False,Hybrid,False,On-going,2 weeks ago,5,False,Full-time,Associate,10000,Food and Beverage Manufacturing
1,"Sr Data Scientist, Operations Research Engineer",PepsiCo,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3266052627/...,True,Hybrid,False,On-going,3 days ago,50,True,Full-time,Mid-Senior level,10000,Food and Beverage Services
2,Senior Data Scientist,PepsiCo,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3270287326/...,True,Hybrid,False,On-going,1 day ago,40,True,Full-time,Mid-Senior level,10000,Food and Beverage Services
3,Data Analyst,TravelPerk,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3341757761/...,False,Hybrid,True,On-going,1 month ago,70,True,Full-time,Mid-Senior level,3000,"Technology, Information and Internet"
4,Frontend Engineer,Scandit,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3219075642/...,False,Remote,True,On-going,3 weeks ago,200,False,Full-time,Mid-Senior level,750,Software Development
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,Data Scientist,Zurich Insurance,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3348244612/...,False,NaN,False,On-going,6 days ago,200,True,Full-time,Mid-Senior level,10000,Insurance
933,Data Scientist,Product Madness 📱🎮,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3350814599/...,False,Hybrid,False,NaN,3 days ago,61,False,Full-time,Entry level,750,Computer Games
934,Data Scientist,Scalapay,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3342334214/...,False,On-site,False,NaN,1 week ago,52,False,Full-time,Entry level,350,Financial Services
935,Data Scientist,SwissBorg,"Barcelona, Catalonia, Spain",https://www.linkedin.com/jobs/view/3171378381/...,False,NaN,False,On-going,1 day ago,26,False,Full-time,Entry level,350,Financial Services
